## The 'IS' bug report:

- Thejasvi Beleyur, October 25 2019

###  A bug in the CPN code was discovered over the weekend of 19-20 October that affected cocktail party nightmare simulations with group sizes larger than 256. The bug showed up as unrealistic 'np.inf' entries in the received level of secondary echoes. This meant that certain secondary echoes had infinite sound pressure level. 

## What caused the 'IS' bug?
The 'is' bug happened because I (TB) misunderstood how the 'is' operator actually works in Python. I treated it like an equals to ('=='). The behaviour of == and 'is' matches well for most numbers - but fails for numbers larger than 256 as I discovered later:

In [1]:
example_numbers = [0,100,255,256,257,300]
example_numbers2 = [0,100,255,256,257,300]

# if 'is' and == showed the same behaviour throughout 
# only (True, True) should be printed out all through the loop
for a,b in zip(example_numbers, example_numbers2):
    print(a is b, a==b)

(True, True)
(True, True)
(True, True)
(True, True)
(False, True)
(False, True)


## Where was the wrong implementation?
The wrong implementation happened in the path assignment routine: 'paths_2daryechoes'. All echoes in the simulations are assigned a 'route' consisting of three numbers in a tuple. Each number represents the index number of a bat, and each echo (primary and secondary) has an emitter, target and receiver. 

### How the path assignment works
Example1. (2,1,2) represents bat #2 calling, the sound reflecting off bat #1 and returning to bat #2. This is a primary echo where the emitter and receiver are the same. 

Example2. (4,1,2) represents bat #4 calling, the sound reflecting off bat #1 and going to bat #2. This is a secondary echo where the emitter and receiver are not the same. 

### Why the infinite sound pressure levels
The problem with path tuples like (1,1,4) is that it implies a zero distance between an emitter and target. The equations to calculate the received level of a sound blow up to infinity when the distance between the emitter and target is zero. 

### The bug:
To generate all secondary echo paths, two for loops were used to iterate over all possible emitters, and all possible targets that could produce sounds given a known receiver. This meant that the last number in the path tuple is fixed:

```
1 def paths_2daryechoes(focal_bat, bats_xy): 
2    # make all emitter-target and target-receiver paths using the row indices as 
3    # an identifier
4    emitters = set(range(bats_xy.shape[0])) - set([focal_bat])
5    targets = set(range(bats_xy.shape[0])) - set([focal_bat])
6    
7    
8    echo_routes = []
9    for an_emitter in emitters:
10        for a_target in targets:
11            if not a_target is an_emitter:
12                    emitter_target_focal = (an_emitter, a_target, focal_bat)
13                    echo_routes.append(emitter_target_focal)
14    return(echo_routes) 
```
Line 11 above makes sure that the target and emitter bats are different. It doesn't make sense for a bat to generate echoes from itself - so we want to disallow that. 

Example3. Wrong path tuple : (1,1,5)
This path tuple has bat #1 emitting, bat #1 as the target and bat #5 as the receiver. It doesn't make sense. 

Line 11 works perfectly fine till the number 256 - and so the moment we have more than 256 bat indices in a group - this check fails - leading to the assignment of proper path tuples (emitter and target different) and improper path tuples (emitter and target the same). 

This has now been fixed as of commit ```ea7f1fa...``` onwards. I also implemented a test that specifically checks that the proper number of expected secondary echeos are generated across group sizes. The current code now reads:
```
1 def paths_2daryechoes(focal_bat, bats_xy): 
2    # make all emitter-target and target-receiver paths using the row indices as 
3    # an identifier
4    emitters = set(range(bats_xy.shape[0])) - set([focal_bat])
5    targets = set(range(bats_xy.shape[0])) - set([focal_bat])
6    
7    
8    echo_routes = []
9    for an_emitter in emitters:
10        for a_target in targets:
11            if a_target!= an_emitter:
12                    emitter_target_focal = (an_emitter, a_target, focal_bat)
13                    echo_routes.append(emitter_target_focal)
14    return(echo_routes) 
```
Line 11 has been changed to check the target-emitter equality in a more stragithforward and reliable way. 

### Checking if simulation outputs are correct now:
While I could isolate what I thought was the bug ... I still don't know if this is truly the case. I ran two simulations yesterday to test this bug. One set of simulations ran a group of 255 and 300 bats for 5 simulations each. Let's load and check out what the simulation outputs look like.

In [2]:
import glob
import sys 
sys.path.append('..//CPN//')
import dill
import numpy as np 


In [3]:
def load_simresult(path_to_simresult):
    '''
    '''
    with open(path_to_simresult, 'rb') as sim:
        output = dill.load(sim)
    return(output)

In [4]:
post_bug_fix_results = glob.glob('post_bug_fix_simresults/*.simresults')

In [5]:
post_bug_fix_results

['post_bug_fix_simresults/testing_fcb94719-e71f-4ee1-b276-e3b615c0c4bc_1067853552_.simresults',
 'post_bug_fix_simresults/testing_477e3128-e3b0-4cf5-b381-7b3f9c03e9c1_209428659_.simresults',
 'post_bug_fix_simresults/testing_a24b7286-0995-4434-810b-bab295daf460_842392783_.simresults',
 'post_bug_fix_simresults/testing_430b179d-dd2c-49a1-9dfc-a06196e9facf_760948913_.simresults',
 'post_bug_fix_simresults/testing_dc29e875-d844-489b-b127-8e9a62629eec_820014527_.simresults',
 'post_bug_fix_simresults/testing_17d427e5-904f-4edd-8bef-a37b02190783_595563844_.simresults',
 'post_bug_fix_simresults/testing_d6e09819-1b84-402b-986c-4b4b34bb6158_227729577_.simresults',
 'post_bug_fix_simresults/testing_ce71d3b0-b32b-4da4-85e4-1b8aaaa8b1e7_161942197_.simresults',
 'post_bug_fix_simresults/testing_6dff7866-1126-458e-93e9-20bfd2ae8455_590908853_.simresults',
 'post_bug_fix_simresults/testing_9649468f-1cac-4a34-93ea-b4abbe8461f6_211872461_.simresults']

In [6]:
def check_all_sounds_have_finite_receivedlevel(simresult_file):
    simoutput = load_simresult(simresult_file)
    siminfo , simdata = simoutput
    echoes_detected, sounds_in_ipi, group_geometry = simdata

    # the group size is number of echoes +1 
    echoes_detected.shape
    # check if there are any anomalous received levels 
    finitelevel_secondary_echoes = np.sum(np.isinf( np.array(sounds_in_ipi['2dary_echoes']['level'])))  == 0
    finitelevel_weird_echoes = np.sum(np.isinf(np.array(sounds_in_ipi['target_echoes']['level']))) == 0
    finitelevel_weird_calls = np.sum(np.isinf(np.array(sounds_in_ipi['conspecific_calls']['level'], dtype='float64'))) ==0

    # check that all of the sounds have finite received levels
    all_sounds_finite = np.all([finitelevel_secondary_echoes,
                                finitelevel_weird_echoes,
                                finitelevel_weird_calls])
    return(all_sounds_finite)

In [7]:
simulation_outputs_normal = []
for each_simresult in post_bug_fix_results:
    simulation_outputs_normal.append(check_all_sounds_have_finite_receivedlevel(each_simresult))

In [8]:
# if everything is as expected all simulation_outputs_normal should be True for each entry
print(simulation_outputs_normal)

[True, True, True, True, True, True, True, True, True, True]


### The 'IS' bug has been fixed, and the recent simulation runs of 255 and 300 bat group sizes show that all received levels are finite. Future simulations should have no issue running with group sizes beyond 256.